# Document Intelligence Deep Dive

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
DOC_INTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOC_INTELLIGENCE_ENDPOINT")
DOC_INTELLIGENCE_KEY = os.getenv("AZURE_DOC_INTELLIGENCE_KEY")
DOC_INTELLIGENCE_VERSION = os.getenv("AZURE_DOC_INTELLIGENCE_VERSION")
MODEL_ID = "prebuilt-document" 
# other values for MODEL_ID 
# - prebuilt-read
# - prebuilt-layout
# - prebuilt-healthInsuranceCard.us
# - prebuilt-tax.us.w2
# - prebuilt-invoice
# - prebuilt-receipt
# - prebuilt-idDocument
# - prebuilt-businessCard
# or you can train your custom model

FEATURES = ['barcodes']
# barcodes        Enable the detection of barcodes in the document.
# formulas        Enable the detection of mathematical expressions in the document.
# keyValuePairs   Enable the detection of general key value pairs (form fields) in the document.
# languages       Enable the detection of the text content language.
# ocrHighResolution  Perform OCR at a higher resolution to handle documents with fine print.
# styleFont       Enable the recognition of various font styles.

In [ ]:
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

credentials = AzureKeyCredential(DOC_INTELLIGENCE_KEY)
client = DocumentAnalysisClient(DOC_INTELLIGENCE_ENDPOINT, credentials)
poller = client.begin_analyze_document(
    MODEL_ID, 
    open('sample_data/CCR Phase 3 Final RFP_20230215.pdf', 'rb'), 
    pages='6-8', 
    # features=FEATURES
)
poller.wait()
result = poller.result()
result

In [ ]:
[l.content for l in result.pages[0].lines]

In [ ]:
for p in result.paragraphs:
    if p.role=='sectionHeading':
        print(f"# {p.content}")
    elif p.role in ['pageFooter','pageHeader','pageNumber']:
        print('__')
    else:
        print(p.content)

In [ ]:
tbl = None 
if len(result.tables) > 0:
    result.tables[0]
    rows = result.tables[0].row_count
    cols = result.tables[0].column_count 
    tbl = [['' for c in range(cols)] for r in range(rows)]
    for cell in result.tables[0].cells:
        tbl[cell.row_index][cell.column_index] = cell.content
tbl

In [ ]:
[l.content for l in result.pages[2].lines]

Let's process a few pages and familiarize ourselves with the outputs.

Feel free to compare the results with Excel files.